In [9]:
from pycorenlp import StanfordCoreNLP
import os
from multiprocessing import Pool
import time
import re

In [54]:
stop_words_dict = ['therapy', 'effect', 'dose', 'dosage', 'week', 'patient', 'long-term', 'short-term', 'data', 
                   'treatment', 'onset', 'frequency', 'study', 'drug', 'reaction', 'trial', 'mm',
                  'program', 'food', 'day', 'time', 'hour', 'reason', 'death', 'physician', 'administration', 
                   'minute', 'msec', 'class', 'doctor', 'nurse', 'year', 'mg', 'take', 'age', 'value', 'average',
                  'ml', 'complication', 'corticosteroid', 'surgery', 'help', 'overdose', 'check', 'report', 'receive',
                  'ast', 'alt']
strip_words_dict = ['follow', 'include', 'im', 'mi']

In [20]:
nlp = StanfordCoreNLP('http://localhost:9000')

In [21]:
drug_name_file = open('valid_drugs_com_web_names.txt','r')
drug_names = drug_name_file.readlines()
drug_names = [drug_name.replace('\n','') for drug_name in drug_names]

In [59]:
def clean_grams(drugname):
    save_filepath = 'drugs/cleaned-grams/' + drugname + '.txt'
    read_filepath = 'drugs/grams/' + drugname + '.txt'
    file = open(read_filepath)
    texts = file.readlines()
    grams = []
    
    for text in texts:
        stop_flag = False
        text = text.rstrip('\n')
        text = re.sub(r'[\W]', ' ', text)
        while True:
            try:
                output = nlp.annotate(text, properties={
                  'annotators': 'lemma',
                  'outputFormat': 'json'
                  })
                break
            except:
                print drugname + ': Failed in parsing...'
                time.sleep(1)
        tokens = output['sentences'][0]['tokens']
        text_lemma = [token['lemma'] for token in tokens]
        for stop_word in stop_words_dict:
            if stop_word in text_lemma:
                stop_flag = True
                break
        if stop_flag:
            continue
        for strip_word in strip_words_dict:
            if strip_word in text_lemma:
                text_lemma.remove(strip_word)
        if text_lemma and text_lemma != ['symptom']:
            grams.append(' '.join(text_lemma))
    
    grams = list(set(grams))
    
    if grams:
        with open(save_filepath, 'w') as file:
            file.write("\n".join(grams))
            file.close()

In [58]:
clean_grams(drug_names[300])

In [49]:
drug_names[300]

'amphotec'